In [2]:
import pandas as pd
import scipy.spatial.distance as sci_dist
import matplotlib
matplotlib.use('QtAgg')
import seaborn as sns
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib
Image.MAX_IMAGE_PIXELS = None
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import poisson
import os
np.random.seed(0)
matplotlib.rcParams['pdf.fonttype'] = 42

plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Arial']

data_folder = '/Users/YaelHeyman/RajLab Dropbox/Yael Heyman/SpatialBarcodes/ImagingData/2024-04-27_spatialbarcodes_SG_expression_mouse_exp/time_zero_output/roi3/exports'
region_data = '/Users/YaelHeyman/RajLab Dropbox/Yael Heyman/SpatialBarcodes/ImagingData/2024-04-27_spatialbarcodes_SG_expression_mouse_exp/time_zero_output/Region_Data_with_Minimal_Values.csv'
cell_by_gene_path = data_folder + '/cell_by_gene_matrix_dilate10_20240718_withbarcodes_atleast3.csv'
transcripts_path = data_folder + '/decode_1_withcoordinates.csv'
coords_path = data_folder + '/segmentation_1_nuclei_cellattributes.csv'
output_folder = data_folder 
cell_by_gene = pd.read_csv(cell_by_gene_path)


In [3]:
pixel2um = 107.11 / 1000000  # Convert pixel to micrometers
combined_df = pd.DataFrame()
region_data = pd.read_csv(region_data)
for j in range(1, 4):  # Adjust range as needed
    # Dynamically construct the path
    data_folder = f"/Users/YaelHeyman/RajLab Dropbox/Yael Heyman/SpatialBarcodes/ImagingData/2024-04-27_spatialbarcodes_SG_expression_mouse_exp/time_zero_output/roi{j}/exports"
    cell_by_gene_path = os.path.join(data_folder, 'cell_by_gene_matrix_dilate10_20240718_withbarcodes_clustering_10bcs_0.4thresh.csv')
    print(data_folder)
    # Load the cell by gene data
    cell_by_gene = pd.read_csv(cell_by_gene_path, index_col=0)
    
    # Get reference X and Y from region_data as scalar values
    ref_x = region_data.loc[region_data['Region'] == f"Region {j}", "Min X (mm)"].iloc[0]
    ref_y = region_data.loc[region_data['Region'] == f"Region {j}", "Min Y (mm)"].iloc[0]
    
    # Correct the coordinates
    cell_by_gene['center_x'] = (cell_by_gene['center_x'] * pixel2um + ref_x)*1000
    cell_by_gene['center_y'] = (cell_by_gene['center_y'] * pixel2um + ref_y)*1000
    
    # Adjust the cell_id to make them unique across regions
    cell_by_gene['cell_id'] = cell_by_gene['cell_id'] + j * 100000
    
    # Concatenate to the combined DataFrame
    combined_df = pd.concat([combined_df, cell_by_gene], ignore_index=True)


/Users/YaelHeyman/RajLab Dropbox/Yael Heyman/SpatialBarcodes/ImagingData/2024-04-27_spatialbarcodes_SG_expression_mouse_exp/time_zero_output/roi1/exports
/Users/YaelHeyman/RajLab Dropbox/Yael Heyman/SpatialBarcodes/ImagingData/2024-04-27_spatialbarcodes_SG_expression_mouse_exp/time_zero_output/roi2/exports
/Users/YaelHeyman/RajLab Dropbox/Yael Heyman/SpatialBarcodes/ImagingData/2024-04-27_spatialbarcodes_SG_expression_mouse_exp/time_zero_output/roi3/exports


In [4]:
barcode_indices = {}
cell_by_gene = combined_df
# Loop through each barcode and add its index to the appropriate list
for index, barcode in enumerate(cell_by_gene['barcode_names']):
    # Convert NaN values to a string representation to handle them as unique keys
    barcode_key = barcode if barcode == barcode else 'NaN'
    if barcode_key in barcode_indices:
        barcode_indices[barcode_key].append(index)
    else:
        barcode_indices[barcode_key] = [index]

# Convert the dictionary to a list of lists (containing the indices)
indices_list = list(barcode_indices.values())
for ids in indices_list:
    print('ids')
    print(ids)
    for id in ids:
        if(id>0):
            print(cell_by_gene.loc[id,"barcode_names"])
unique_index_list = [
    cell_by_gene.loc[ids[0], ["barcode_names", "n_called_barcodes"]]
    for ids in indices_list
    if pd.notna(cell_by_gene.loc[ids[0], "barcode_names"])
]
unique_index_df = pd.DataFrame(unique_index_list, columns=["barcode_names", "n_called_barcodes"])
# Now you can access the column:
unique_index_df["barcode_names"]
print('Percentage zero barcodes: ' + str(np.sum(pd.isna(cell_by_gene['barcode_names'])) / len(cell_by_gene)))


ids
[0]
ids
[1, 34832, 36525]
bc_070-bc_085
bc_070-bc_085
bc_070-bc_085
ids
[2, 50, 286]
bc_010-bc_040-bc_057-bc_085
bc_010-bc_040-bc_057-bc_085
bc_010-bc_040-bc_057-bc_085
ids
[3, 30327]
bc_001-bc_045
bc_001-bc_045
ids
[4]
bc_009-bc_038-bc_092
ids
[5, 375, 14420]
bc_082-bc_088
bc_082-bc_088
bc_082-bc_088
ids
[6, 126, 225, 1070, 1591, 1599, 2092, 2324, 2462, 2920, 5507, 5883, 6721, 7714, 7811, 9500, 9611, 9789, 9982, 10437, 10462, 11257, 12227, 12483, 12945, 13881, 17210, 17296, 17746, 19442, 19491, 20178, 21223, 21265, 22482, 24529, 27245, 27382, 28253, 28720, 29479, 29501, 29933, 33532, 34228, 35367, 35369, 35562, 35586, 37986, 38038, 39574, 39581, 39680, 39865, 40565]
bc_065
bc_065
bc_065
bc_065
bc_065
bc_065
bc_065
bc_065
bc_065
bc_065
bc_065
bc_065
bc_065
bc_065
bc_065
bc_065
bc_065
bc_065
bc_065
bc_065
bc_065
bc_065
bc_065
bc_065
bc_065
bc_065
bc_065
bc_065
bc_065
bc_065
bc_065
bc_065
bc_065
bc_065
bc_065
bc_065
bc_065
bc_065
bc_065
bc_065
bc_065
bc_065
bc_065
bc_065
bc_065
bc_06

In [5]:
1-np.sum(pd.isna(cell_by_gene['barcode_names'])) / len(cell_by_gene)

np.float64(0.891052185494612)

In [6]:
data =unique_index_df["n_called_barcodes"]  #if using the grouping logic
# data = cell_by_gene["n_called_barcodes"] #if going for direct count
# Estimate the lambda (mean) of the Poisson distribution
lambda_estimate = np.mean(data)
lambda_estimate 
# Calculate the empirical PMF
unique_values, counts = np.unique(data, return_counts=True)
empirical_pmf = counts / len(data)

# Theoretical Poisson PMF
x_values = np.arange(0, max(data) + 1)  # Range of possible values
theoretical_pmf = poisson.pmf(x_values, mu=lambda_estimate)

# Plotting the PMF
plt.figure(figsize=(2,2))

# Empirical PMF as a bar plot
plt.bar(unique_values, empirical_pmf, width=0.4, label='Empirical PMF', alpha=0.7, align='center')

# Theoretical Poisson PMF as a line plot
plt.plot(x_values, theoretical_pmf, 'go-', label=f'Theoretical Poisson PMF (λ={lambda_estimate:.1f})')
plt.xlim([-1,16])
# Labels and legend
plt.xlabel('Number of barcodes per cell')
plt.ylabel('Probability')
plt.title('Empirical vs theoretical PDF')
plt.legend(bbox_to_anchor=(1.05, 0.5),
          loc='center left',
          borderaxespad=0)
plt.grid(True)
plt.savefig(output_folder + '/empirical vs predicted pdf time zero unique bc.pdf',bbox_inches='tight')
plt.show()


In [7]:
cdf = np.empty(0)
for i in range(len(unique_values)):
    cdf = np.append(cdf, np.sum(empirical_pmf[i:]))
cdf_df = pd.DataFrame({ 'num barcodes': unique_values,'cdf': cdf}) 
cdf_df.to_csv(os.path.join(output_folder, 'cdf vs unique number of barcodes.csv'))
pdf_df = pd.DataFrame({ 'num barcodes': unique_values,'pdf': empirical_pmf})
pdf_df.to_csv(os.path.join(output_folder, 'pdf vs unique number of barcodes.csv'))